In [ ]:
!pip install gensim
!pip install nltk

In [ ]:
import os
import pandas as pd
import numpy as np
import string
# from operator import itemgetter
from collections import Counter, OrderedDict

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize

sample_text = "This is a test. Let's see if this works."
print(sent_tokenize(sample_text))


['This is a test.', "Let's see if this works."]


In [ ]:
import kagglehub

# Download dataset
path = kagglehub.dataset_download("zynicide/wine-reviews")
print("Path to dataset files:", path)

# List files in the dataset directory
print(os.listdir(path))

Path to dataset files: /root/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4
['winemag-data-130k-v2.json', 'winemag-data-130k-v2.csv', 'winemag-data_first150k.csv']


First, import the wine dataset.

In [ ]:
# Path to the CSV file
csv_path = os.path.join(path, "winemag-data-130k-v2.csv")

# Load the CSV file
df_wine = pd.read_csv(csv_path)

# Preview the data
print(df_wine.head())
print(df_wine.columns)
print(df_wine.info())
print(df_wine.isnull().sum())

   Unnamed: 0   country                                        description  \
0           0     Italy  Aromas include tropical fruit, broom, brimston...   
1           1  Portugal  This is ripe and fruity, a wine that is smooth...   
2           2        US  Tart and snappy, the flavors of lime flesh and...   
3           3        US  Pineapple rind, lemon pith and orange blossom ...   
4           4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

              region_1           region_2         taster_name  \
0              

Then, the food dataset.

In [ ]:
# Download latest version
path2 = kagglehub.dataset_download("snap/amazon-fine-food-reviews")

print("Path to dataset files:", path2)

In [ ]:
# Path to the CSV file
csv_path2 = os.path.join(path2, "Reviews.csv")

# Load the CSV file
df_food = pd.read_csv(csv_path2)

# Preview the data
print(df_food.head())
print(df_food.columns)
print(df_food.info())
print(df_food.isnull().sum())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

### 1. Training our Word Embeddings

First, we need to train a Word2Vec model on all the words in our corpus. We will process our wine and food terms separately - some of the wine terms will be standardized to account for commonalities in the colorful language of the world of wine.

In [ ]:
wine_reviews_list = list(df_wine['description'])
food_reviews_list = list(df_food['Text'])

To begin, we need to tokenize the terms in our corpus (wine and food).

In [ ]:
full_wine_reviews_list = [str(r) for r in wine_reviews_list]
full_wine_corpus = ' '.join(full_wine_reviews_list)
wine_sentences_tokenized = sent_tokenize(full_wine_corpus)

full_food_reviews_list = [str(r) for r in food_reviews_list]
full_food_corpus = ' '.join(full_food_reviews_list)
food_sentences_tokenized = sent_tokenize(full_food_corpus)

print(wine_sentences_tokenized[:2])
print(food_sentences_tokenized[:2])

['Aromas include tropical fruit, broom, brimstone and dried herb.', "The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."]
['I have bought several of the Vitality canned dog food products and have found them all to be of good quality.', 'The product looks more like a stew than a processed meat and it smells better.']


Next, the text in each sentence is normalized (tokenize, remove punctuation and remove stopwords).

In [ ]:
stop_words = set(stopwords.words('english'))

punctuation_table = str.maketrans({key: None for key in string.punctuation})
sno = SnowballStemmer('english')

def normalize_text(raw_text):
    try:
        word_list = word_tokenize(raw_text)
        normalized_sentence = []
        for w in word_list:
            try:
                w = str(w)
                lower_case_word = str.lower(w)
                stemmed_word = sno.stem(lower_case_word)
                no_punctuation = stemmed_word.translate(punctuation_table)
                if len(no_punctuation) > 1 and no_punctuation not in stop_words:
                    normalized_sentence.append(no_punctuation)
            except:
                continue
        return normalized_sentence
    except:
        return ''

normalized_wine_sentences = []
for s in wine_sentences_tokenized:
    normalized_text = normalize_text(s)
    normalized_wine_sentences.append(normalized_text)

normalized_food_sentences = []
for s in food_sentences_tokenized:
    normalized_text = normalize_text(s)
    normalized_food_sentences.append(normalized_text)

Not all of the terms we are interested in are single words. Some of the terms are phrases, consisting of two (or more!) words. An example of this might be 'high tannin'. We can use gensim's Phrases feature to extract all the most relevant bi- and tri-grams from our corpus.

We will train a separate trigram model for wine and for food.

In [ ]:
# first, take care of the wine trigrams
 wine_bigram_model = Phrases(normalized_wine_sentences, min_count=100)
wine_bigrams = [wine_bigram_model[line] for line in normalized_wine_sentences]
wine_trigram_model = Phrases(wine_bigrams, min_count=50)
phrased_wine_sentences = [wine_trigram_model[line] for line in wine_bigrams]
 wine_trigram_model.save('wine_trigrams.pkl')

### now, do the same for food
food_bigram_model = Phrases(normalized_food_sentences, min_count=100)
food_bigrams = [food_bigram_model[sent] for sent in normalized_food_sentences]
food_trigram_model = Phrases(food_bigrams, min_count=50)
phrased_food_sentences = [food_trigram_model[sent] for sent in food_bigrams]
food_trigram_model.save('food_trigrams.pkl')

If the trigram model has already been trained, simply retrieve it.

In [ ]:
wine_trigram_model = Phraser.load('wine_trigrams.pkl')
food_trigram_model = Phraser.load('food_trigrams.pkl')

Now for the most important part: leveraging existing wine theory, the work of others like Bernard Chen, wine descriptor mappings and the UC Davis wine wheel, the top 5000 most frequent wine terms were reviewed to (i) determine whether they are a descriptor that can be derived by blind tasting, and (ii) whether they are informative (judgments like 'tasty' and 'great' are not considered to be informative). The roughly 1000 descriptors that remain were then mapped onto a normalized descriptor, a category and a class:

In [ ]:
descriptor_mapping = pd.read_csv('descriptor_mapping.csv', encoding='latin1').set_index('raw descriptor')

def return_mapped_descriptor(word, mapping):
    if word in list(mapping.index):
        normalized_word = mapping.at[word, 'level_3']
        return normalized_word
    else:
        return word

normalized_wine_sentences = []
for sent in phrased_wine_sentences:
    normalized_wine_sentence = []
    for word in sent:
        normalized_word = return_mapped_descriptor(word, descriptor_mapping)
        normalized_wine_sentence.append(str(normalized_word))
    normalized_wine_sentences.append(normalized_wine_sentence)

We will go through the same process for food, but without normalizing the nonaroma descriptors.

In [ ]:
aroma_descriptor_mapping = descriptor_mapping.loc[descriptor_mapping['type'] == 'aroma']
normalized_food_sentences = []
for sent in phrased_food_sentences:
    normalized_food_sentence = []
    for word in sent:
        normalized_word = return_mapped_descriptor(word, aroma_descriptor_mapping)
        normalized_food_sentence.append(str(normalized_word))
    normalized_food_sentences.append(normalized_food_sentence)

Now, let's combine the wine dataset with our food dataset so we can train our embeddings. We want to make sure that the food and wine embeddings are calculated in the same feature space so that we can compute similarity vectors later on.

In [ ]:
normalized_sentences = normalized_wine_sentences + normalized_food_sentences

We are ready to train our Word2Vec model!

In [23]:
wine_word2vec_model = Word2Vec(normalized_sentences, size=300, min_count=8, iter=15)
print(wine_word2vec_model)

wine_word2vec_model.save('food_word2vec_model.bin')

Word2Vec<vocab=36695, vector_size=300, alpha=0.025>


In [24]:
# if the word2vec model has already been trained, simply load it
wine_word2vec_model = Word2Vec.load("food_word2vec_model.bin")

### 2. Preprocessing our Wine Dataset

We can now turn our attention to our wine dataset. Descriptions for a single wine are unlikely to contain sufficient information about all the nonaromas and aromas to yield consistent and reliable pairing recommendations. As such, we will produce recommendations at the grape variety & subregion level.

First, let's normalize the names of the grape varieties in our dataset.

In [25]:
# Display the unique values in the 'variety' column
unique_varieties = df_wine['variety'].unique()
print(f"Unique grape varieties ({len(unique_varieties)} total):")
print(unique_varieties)

Unique grape varieties (708 total):
['White Blend' 'Portuguese Red' 'Pinot Gris' 'Riesling' 'Pinot Noir'
 'Tempranillo-Merlot' 'Frappato' 'Gewürztraminer' 'Cabernet Sauvignon'
 'Nerello Mascalese' 'Chardonnay' 'Malbec' 'Tempranillo Blend' 'Meritage'
 'Red Blend' 'Merlot' "Nero d'Avola" 'Chenin Blanc' 'Gamay'
 'Sauvignon Blanc' 'Viognier-Chardonnay' 'Primitivo' 'Catarratto'
 'Inzolia' 'Petit Verdot' 'Monica' 'Bordeaux-style White Blend' 'Grillo'
 'Sangiovese' 'Cabernet Franc' 'Champagne Blend'
 'Bordeaux-style Red Blend' 'Aglianico' 'Petite Sirah' 'Touriga Nacional'
 'Carmenère' 'Albariño' 'Petit Manseng' 'Rosé' 'Zinfandel' 'Vernaccia'
 'Rosato' 'Grüner Veltliner' 'Viognier' 'Vermentino' 'Grenache Blanc'
 'Syrah' 'Nebbiolo' 'Shiraz-Cabernet Sauvignon' 'Pinot Blanc'
 'Alsace white blend' 'Barbera' 'Rhône-style Red Blend' 'Portuguese White'
 'Graciano' 'Tannat-Cabernet' 'Sauvignon' 'Sangiovese Grosso' 'Torrontés'
 'Prugnolo Gentile' 'G-S-M' 'Verdejo' 'Fumé Blanc' 'Furmint'
 'Pinot Bianco'

In [62]:
variety_mapping = {
    # Whites
    'Pinot Gris': 'Pinot Grigio',
    'Pinot Grigio/Gris': 'Pinot Grigio',
    'Grüner Veltliner': 'Gruner Veltliner',
    'Fumé Blanc': 'Sauvignon Blanc',
    'Garganega': 'Soave',
    'Verdejo-Viura': 'Verdejo',
    'Riesling-Chardonnay': 'White Blend',
    'Sauvignon Blanc-Semillon': 'White Bordeaux Blend',
    'Semillon-Sauvignon Blanc': 'White Bordeaux Blend',
    'Trebbiano Spoletino': 'Trebbiano',
    'Trebbiano di Lugana': 'Trebbiano',
    'Malvasia Bianca': 'Malvasia',
    'Verdelho': 'Verdelho',
    'Picpoul': 'Piquepoul',
    'Alvarinho': 'Albarino',
    'Verdicchio': 'Verdicchio',
    'Marsanne-Roussanne': 'Rhone White Blend',
    'Chardonnay-Sauvignon Blanc': 'White Blend',
    'Sauvignon Blanc-Chenin Blanc': 'White Blend',
    'Chenin Blanc-Chardonnay': 'White Blend',
    'Viognier-Chardonnay': 'White Blend',
    'Grenache Blanc': 'Rhone White Blend',
    'Assyrtiko': 'Assyrtiko',
    'Müller-Thurgau': 'Muller-Thurgau',
    'Sylvaner': 'Silvaner',
    'Zibibbo': 'Muscat of Alexandria',
    'Muscat Blanc à Petits Grains': 'Muscat',
    'Prosecco': 'Glera',
    'Pinot Bianco': 'Pinot Blanc',
    'Sémillon': 'Semillon',

    # Reds
    'Shiraz': 'Syrah',
    'Syrah-Grenache': 'Rhone Red Blend',
    'Grenache-Syrah': 'Rhone Red Blend',
    'Garnacha': 'Grenache',
    'Cabernet Sauvignon-Merlot': 'Bordeaux Blend',
    'Merlot-Cabernet Sauvignon': 'Bordeaux Blend',
    'Petit Verdot': 'Petit Verdot',
    'Tempranillo-Cabernet Sauvignon': 'Tempranillo Blend',
    'Malbec-Cabernet Franc': 'Bordeaux Blend',
    'Tinta del Pais': 'Tempranillo',
    'Tinta Fina': 'Tempranillo',
    'Aragonês': 'Tempranillo',
    'Cabernet Sauvignon-Syrah': 'Cabernet-Syrah Blend',
    'Cabernet Sauvignon-Carmenère': 'Cabernet-Carmenere Blend',
    'Monastrell': 'Mourvedre',
    'Zinfandel': 'Primitivo',
    'Blaufränkisch': 'Blaufrankisch',
    'Pinot Nero': 'Pinot Noir',
    'Spätburgunder': 'Pinot Noir',
    'Ribolla Gialla': 'Ribolla Gialla',
    'Frappato': 'Frappato',
    'Nero d\'Avola': 'Nero d\'Avola',
    'Aglianico': 'Aglianico',
    'Barbera-Nebbiolo': 'Barbera',
    'Cesanese d\'Affile': 'Cesanese',
    'Lagrein': 'Lagrein',

    # Sparkling
    'Champagne Blend': 'Champagne',
    'Sparkling Blend': 'Sparkling Wine',
    'Portuguese Sparkling': 'Sparkling Wine',

    # Rosés
    'Rosé': 'Rose',
    'Rosado': 'Rose',
    'Portuguese Rosé': 'Rose',

    # Fortified and Sweet Wines
    'Sherry': 'Sherry',
    'Port': 'Port',
    'Madeira Blend': 'Madeira',
    'Pedro Ximénez': 'Pedro Ximenez',
    'Moscatel de Alejandría': 'Muscat of Alexandria',
    'Tokaji': 'Tokaji',

    # Others and Rare Varieties
    'Roussanne': 'Rhone White Blend',
    'Marsanne': 'Rhone White Blend',
    'Carmenère': 'Carmenere',
    'Albariño': 'Albarino',
    'Gewürztraminer': 'Gewurztraminer',
    'Vermentino': 'Vermentino',
    'Viognier': 'Viognier',
    'Cortese': 'Cortese (Gavi)',
    'Nerello Mascalese': 'Nerello Mascalese',
    'Dolcetto': 'Dolcetto',
    'Cinsault': 'Cinsault',
    'Carignan': 'Carignan',
    'Savagnin': 'Savagnin',
    'Tannat': 'Tannat',
    'Malbec': 'Malbec',
    'Petit Manseng': 'Petit Manseng',
    'Grenache': 'Grenache',
    'Pinotage': 'Pinotage',
    'Negroamaro': 'Negroamaro',
    'Falanghina': 'Falanghina',
    'Vernaccia': 'Vernaccia',
    'Primitivo': 'Zinfandel',
    'Cabernet Franc': 'Cabernet Franc',
    'Cabernet Sauvignon': 'Cabernet Sauvignon',
    'Chardonnay': 'Chardonnay',
    'Merlot': 'Merlot',
    'Sangiovese': 'Sangiovese',
    'Nebbiolo': 'Nebbiolo',
    'Gamay': 'Gamay',
}

def consolidate_varieties(variety_name):
    if variety_name in variety_mapping:
        return variety_mapping[variety_name]
    else:
        return variety_name


df_wine_clean = df_wine.copy()
df_wine_clean['variety'] = df_wine_clean['variety'].apply(lambda x: variety_mapping.get(x, x))
df_wine_clean.rename(columns={'region_1': 'Subregion', 'region_2': 'Region'}, inplace=True)

Next, we need to define the set of geography subregions we will use to define our wines. Not too general, not too specific... just right.

In [63]:
# Check the updated column names
print(df_wine_clean.columns)

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'Subregion', 'Region', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')


In [41]:
order_of_geographies = ['Subregion', 'Region', 'province', 'country']

# Replace NaN and invalid values with 'none'
def replace_nan_for_zero(value):
    if str(value).lower() in ['0', 'nan', 'none']:
        return 'none'
    else:
        return value

for o in order_of_geographies:
    df_wine_clean[o] = df_wine_clean[o].apply(replace_nan_for_zero)

# Verify there are no NaN values
print(df_wine_clean[order_of_geographies].isnull().sum())

Subregion    0
Region       0
province     0
country      0
dtype: int64


In [43]:
print(df_wine_clean.columns)

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'Subregion', 'Region', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')


In [64]:
df_wine_clean['geo_normalized'] = (
    df_wine_clean['Subregion'] + ', ' +
    df_wine_clean['Region'] + ', ' +
    df_wine_clean['province'] + ', ' +
    df_wine_clean['country']
)

In [65]:
# Group by 'Variety', 'Country', 'Province', 'Region', and 'Subregion', and count occurrences
variety_geo = df_wine_clean.groupby(['variety', 'country', 'province', 'Region', 'Subregion']).size().reset_index(name='count')

# Filter for groups where count > 1
variety_geo_sliced = variety_geo.loc[variety_geo['count'] > 1]

# Create a new DataFrame with the relevant columns
vgeos_df = pd.DataFrame(variety_geo_sliced, columns=['variety', 'country', 'province', 'Region', 'Subregion', 'count'])

# Save to CSV
vgeos_df.to_csv('varieties_all_geos.csv', index=False)

# Preview the result
print(vgeos_df.head())

       variety country          province Region            Subregion  count
0     Abouriou  France  Southwest France   none  Côtes du Marmandais      2
4  Agiorgitiko  Greece           Corinth   none                 none      2
5  Agiorgitiko  Greece             Nemea   none                 none     51
7  Agiorgitiko  Greece       Peloponnese   none                 none      7
8    Aglianico   Italy       Italy Other   none        Vino Spumante      2


In [45]:
import os
print(os.getcwd())

/content


In [46]:
print(os.listdir())

['.config', 'food_word2vec_model.bin.syn1neg.npy', 'drive', 'wine_trigrams.pkl', 'varieties_all_geos.csv', 'food_word2vec_model.bin.wv.vectors.npy', 'food_trigrams.pkl', 'food_word2vec_model.bin', 'sample_data']


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# Define the path to the wine_food_pairing folder
file_path = '/content/drive/My Drive/Final Project/wine_food_pairing/varieties_all_geos.csv'

# Save the file to the specified folder
vgeos_df.to_csv(file_path, index=False)

print(f"File saved to: {file_path}")

File saved to: /content/drive/My Drive/Final Project/wine_food_pairing/varieties_all_geos.csv


In [66]:
df_wine_merged = pd.merge(
    left=df_wine_clean,
    right=vgeos_df,
    left_on=['variety', 'country', 'province', 'Region', 'Subregion'],
    right_on=['variety', 'country', 'province', 'Region', 'Subregion']
)

# Drop unnecessary columns
columns_to_drop = [
    'Unnamed: 0', 'designation', 'price', 'province', 'Region',
    'Subregion', 'taster_name', 'taster_twitter_handle', 'winery', 'count'
]
df_wine_merged.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Verify the resulting shape
print("Merged DataFrame shape:", df_wine_merged.shape)

Merged DataFrame shape: (126612, 6)


We only want to keep wine types (location + variety) that appear frequently enough in our dataset.

In [59]:
print(df_wine_merged.columns)

Index(['country', 'description', 'points', 'title', 'variety'], dtype='object')


In [67]:
variety_geos = df_wine_merged.groupby(['variety', 'geo_normalized']).size()
at_least_n_types = variety_geos[variety_geos > 30].reset_index()
df_wine_merged_filtered = pd.merge(df_wine_merged, at_least_n_types, left_on=['variety', 'geo_normalized'], right_on=['variety', 'geo_normalized'])
df_wine_merged_filtered = df_wine_merged_filtered[['title', 'variety', 'geo_normalized', 'description']]
print(df_wine_merged_filtered.shape)

(92514, 4)


Now, we will extract 7 vectors for every wine:

- aroma vector (the aggregate of all the aroma descriptors in a wine)
- nonaroma vectors (an aggregate vector for only aroma & non-aroma descriptors matching the core tastes below):
    - sweetness
    - acid
    - salt
    - piquant
    - fat
    - bitter
    
 In our descriptor file, we have defined which normalized descriptors pertain to each nonaroma.

In [68]:
file_path2 = '/content/drive/My Drive/Final Project/wine_food_pairing/descriptor_mapping_tastes.csv'

In [69]:
wine_reviews = list(df_wine_merged_filtered['description'])

descriptor_mapping = pd.read_csv(file_path2, encoding='latin1').set_index('raw descriptor')

core_tastes = ['aroma', 'weight', 'sweet', 'acid', 'salt', 'piquant', 'fat', 'bitter']
descriptor_mappings = dict()
for c in core_tastes:
    if c=='aroma':
        descriptor_mapping_filtered=descriptor_mapping.loc[descriptor_mapping['type']=='aroma']
    else:
        descriptor_mapping_filtered=descriptor_mapping.loc[descriptor_mapping['primary taste']==c]
    descriptor_mappings[c] = descriptor_mapping_filtered


def return_descriptor_from_mapping(descriptor_mapping, word, core_taste):
    if word in list(descriptor_mapping.index):
        descriptor_to_return = descriptor_mapping['combined'][word]
        return descriptor_to_return
    else:
        return None

review_descriptors = []
for review in wine_reviews:
    taste_descriptors = []
    normalized_review = normalize_text(review)
    phrased_review = wine_trigram_model[normalized_review]
#     print(phrased_review)

    for c in core_tastes:
        descriptors_only = [return_descriptor_from_mapping(descriptor_mappings[c], word, c) for word in phrased_review]
        no_nones = [str(d).strip() for d in descriptors_only if d is not None]
        descriptorized_review = ' '.join(no_nones)
        taste_descriptors.append(descriptorized_review)
    review_descriptors.append(taste_descriptors)


Now we will take the list of descriptors for each wine and its aroma/nonaroma vectors and compute a TF-IDF weighted embedding for each. We will store the results in a dataframe.

In [72]:
taste_descriptors = []
taste_vectors = []

for n, taste in enumerate(core_tastes):
    print(taste)
    taste_words = [r[n] for r in review_descriptors]

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit(taste_words)
    dict_of_tfidf_weightings = dict(zip(X.get_feature_names_out(), X.idf_))

    wine_review_descriptors = []
    wine_review_vectors = []

    for d in taste_words:
        descriptor_count = 0
        weighted_review_terms = []
        terms = d.split(' ')
        for term in terms:
            if term in dict_of_tfidf_weightings.keys():
                tfidf_weighting = dict_of_tfidf_weightings[term]
                try:
                    word_vector = wine_word2vec_model.wv.get_vector(term).reshape(1, 300)
                    weighted_word_vector = tfidf_weighting * word_vector
                    weighted_review_terms.append(weighted_word_vector)
                    descriptor_count += 1
                except:
                    continue
            else:
                continue
        try:
            review_vector = sum(weighted_review_terms)/len(weighted_review_terms)
            review_vector = review_vector[0]
        except:
            review_vector = np.nan
#         terms_and_vec = [terms, review_vector]
        wine_review_vectors.append(review_vector)
        wine_review_descriptors.append(terms)

    taste_vectors.append(wine_review_vectors)
    taste_descriptors.append(wine_review_descriptors)


taste_vectors_t = list(map(list, zip(*taste_vectors)))
taste_descriptors_t = list(map(list, zip(*taste_descriptors)))

df_review_vecs = pd.DataFrame(taste_vectors_t, columns=core_tastes)

columns_taste_descriptors = [a + '_descriptors' for a in core_tastes]
df_review_descriptors = pd.DataFrame(taste_descriptors_t, columns=columns_taste_descriptors)

df_wine_vecs = pd.concat([df_wine_merged_filtered, df_review_descriptors, df_review_vecs], axis=1)
df_wine_vecs.head(5)

aroma
weight
sweet
acid
salt
piquant
fat
bitter


,title,variety,geo_normalized,description,aroma_descriptors,weight_descriptors,sweet_descriptors,acid_descriptors,salt_descriptors,piquant_descriptors,fat_descriptors,bitter_descriptors,aroma,weight,sweet,acid,salt,piquant,fat,bitter
0,Nicosia 2013 Vulkà Bianco (Etna),White Blend,"Etna, none, Sicily & Sardinia, Italy","Aromas include tropical fruit, broom, brimston...","[tropical_fruit, fruit, herb, apple, citrus, s...",[],"[dry, dry]",[high_acid],[],[],[],[],"[4.3810487, -0.3706384, 1.4901603, -3.0812416,...",NaN,"[4.605947, -2.5161955, 3.8138943, -1.7038205, ...","[0.36383158, 0.8201832, 0.88234586, 0.26689127...",NaN,NaN,NaN,NaN
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,"none, none, Douro, Portugal","This is ripe and fruity, a wine that is smooth...","[ripe, fruit, juicy, berry, fruit]",[],[],[high_acid],[],[],[],"[low_tannin, high_tannin]","[2.0581026, 2.7096438, 2.2034063, -3.0944724, ...",NaN,NaN,"[0.36383158, 0.8201832, 0.88234586, 0.26689127...",NaN,NaN,NaN,NaN
2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Grigio,"Willamette Valley, Willamette Valley, Oregon, US","Tart and snappy, the flavors of lime flesh and...","[lime, rind, pineapple, steel]",[],[],"[high_acid, high_acid, high_acid]",[],[],[],[],"[8.45243, -2.2845829, 3.6278782, -3.4782786, -...",NaN,NaN,"[0.3638316, 0.8201832, 0.88234586, 0.26689127,...",NaN,NaN,NaN,NaN
3,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,"Willamette Valley, Willamette Valley, Oregon, US","Much like the regular bottling from 2012, this...","[earth, herb]",[full_bodied],[],[],[],[],[],"[high_tannin, high_tannin, high_tannin]","[-1.4828606, 3.7593317, 3.4948366, -3.4251862,...","[0.27407643, -0.39012954, 2.399533, -3.3532321...",NaN,NaN,NaN,NaN,NaN,NaN
4,Trimbach 2012 Gewurztraminer (Alsace),Gewurztraminer,"Alsace, none, Alsace, France",This dry and restrained wine offers spice in p...,[spice],[],[dry],[],[],[],[],[high_tannin],"[-0.16107632, -2.5843647, 1.4369718, 1.3805469...",NaN,"[4.605947, -2.5161955, 3.8138943, -1.7038205, ...",NaN,NaN,NaN,NaN,NaN


If we don't have a nonaroma embedding for one of the wines, we will simply take the average nonaroma embedding for all the wines in the dataset.

In [73]:
# pull the average embedding for the wine attribute across all wines.
avg_taste_vecs = dict()
for t in core_tastes:
    # look at the average embedding for a taste, across all wines that have descriptors for that taste
    review_arrays = df_wine_vecs[t].dropna()
    average_taste_vec = np.average(review_arrays)
    avg_taste_vecs[t] = average_taste_vec

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Now, let's find the average embedding for each type of wine (aromas and all nonaromas). We have defined the different types of wines by grape variety and geography, keeping only those with a sufficiently large sample size.

For each variety, we will pull (i) a 300-dimensional aroma vector, and (ii) 7 non-aroma scalars.

In [75]:
print(df_wine_vecs.columns)

Index(['title', 'variety', 'geo_normalized', 'description',
       'aroma_descriptors', 'weight_descriptors', 'sweet_descriptors',
       'acid_descriptors', 'salt_descriptors', 'piquant_descriptors',
       'fat_descriptors', 'bitter_descriptors', 'aroma', 'weight', 'sweet',
       'acid', 'salt', 'piquant', 'fat', 'bitter'],
      dtype='object')


In [76]:
normalized_geos = list(set(zip(df_wine_vecs['variety'], df_wine_vecs['geo_normalized'])))

def subset_wine_vectors(list_of_varieties, wine_attribute):
    wine_variety_vectors = []
    for v in list_of_varieties:

        one_var_only = df_wine_vecs.loc[(df_wine_vecs['variety'] == v[0]) &
                                                (df_wine_vecs['geo_normalized'] == v[1])]
        if len(list(one_var_only.index)) < 1 or str(v[1][-1]) == '0':
            continue
        else:
            taste_vecs = list(one_var_only[wine_attribute])
            taste_vecs = [avg_taste_vecs[wine_attribute] if 'numpy' not in str(type(x)) else x for x in taste_vecs]
            average_variety_vec = np.average(taste_vecs, axis=0)

            descriptor_colname = wine_attribute + '_descriptors'
            all_descriptors = [i[0] for i in list(one_var_only[descriptor_colname])]
            word_freqs = Counter(all_descriptors)
            most_common_words = word_freqs.most_common(50)
            top_n_words = [(i[0], "{:.2f}".format(i[1]/len(taste_vecs))) for i in most_common_words]
            top_n_words = [i for i in top_n_words if len(i[0])>2]
            wine_variety_vector = [v, average_variety_vec, top_n_words]

            wine_variety_vectors.append(wine_variety_vector)

    return wine_variety_vectors


def pca_wine_variety(list_of_varieties, wine_attribute, pca=True):
    wine_var_vectors = subset_wine_vectors(normalized_geos, wine_attribute)

    wine_varieties = [str(w[0]).replace('(', '').replace(')', '').replace("'", '').replace('"', '') for w in wine_var_vectors]
    wine_var_vec = [w[1] for w in wine_var_vectors]
    if pca:
        pca = PCA(1)
        wine_var_vec = pca.fit_transform(wine_var_vec)
        wine_var_vec = pd.DataFrame(wine_var_vec, index=wine_varieties)
    else:
        wine_var_vec = pd.Series(wine_var_vec, index=wine_varieties)
    wine_var_vec.sort_index(inplace=True)

    wine_descriptors = pd.DataFrame([w[2] for w in wine_var_vectors], index=wine_varieties)
    wine_descriptors = pd.melt(wine_descriptors.reset_index(), id_vars='index')
    wine_descriptors.sort_index(inplace=True)

    return wine_var_vec, wine_descriptors

taste_dataframes = []
# generate the dataframe of aromas vectors as output,
aroma_vec, aroma_descriptors = pca_wine_variety(normalized_geos, 'aroma', pca=False)
taste_dataframes.append(aroma_vec)

# generate the dataframes of nonaroma scalars
for tw in core_tastes[1:]:
    pca_w_dataframe, nonaroma_descriptors = pca_wine_variety(normalized_geos, tw, pca=True)
    taste_dataframes.append(pca_w_dataframe)

# combine all the dataframes created above into one
all_nonaromas = pd.concat(taste_dataframes, axis=1)
all_nonaromas.columns = core_tastes

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan 

In [ ]:
# save the 50 top descriptors for each wine variety as a CSV file. We will us this later to dig deeper into our proposed wine recommendations.

aroma_descriptors_copy = aroma_descriptors.copy()
aroma_descriptors_copy.set_index('index', inplace=True)
aroma_descriptors_copy.dropna(inplace=True)

aroma_descriptors_copy = pd.DataFrame(aroma_descriptors_copy['value'].tolist(), index=aroma_descriptors_copy.index)
aroma_descriptors_copy.columns = ['descriptors', 'relative_frequency']
aroma_descriptors_copy.to_csv('wine_variety_descriptors.csv')

At the moment, it's hard to interpret the nonaroma scalars. To allow for greater interpretability, we will normalize the nonaroma scalars between 0 and 1.

In [ ]:
def normalize(df, cols_to_normalize):
    for feature_name in cols_to_normalize:
        print(feature_name)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        df[feature_name] = df[feature_name].apply(lambda x: (x- min_value)/(max_value-min_value))
#         (df[feature_name] - min_value) / (max_value - min_value)
    return df

all_nonaromas_normalized = normalize(all_nonaromas, cols_to_normalize=core_tastes[1:])
all_nonaromas_normalized.to_csv('wine_aromas_nonaromas.csv')

### 3. Preparing our Food Dataset

Now that we have our wine aroma vectors and the nonaroma scalars, we can turn our attention to food.

We will want to generate nonaroma vectors for any type of food that we want a wine pairing with. For food, we don't have the luxury of being able to define nonaroma vs. aroma descriptors, so the approach we take will be slightly different:

The aroma vector will be the full food embedding.

We will define an embedding for each of our core nonaromas (sweet, acid, salt, piquant, fat and bitter), and the weight/body of the food. We will define the maximum distance between each of the nonaroma embeddings and a range of commonly appearing foods. The foods that least and most resemble each nonaroma will eventually allow us to create a normalized scale between 0 (very dissimilar) and 1 (very similar) to say how much a food reflects each nonaroma.

First, let's load this list of common foods.

In [ ]:
foods = pd.read_csv('list_of_foods.csv')
foods_list = list(foods['Food'])
foods_list_normalized = [normalize_text(f) for f in foods_list]
foods_list_preprocessed = [food_trigram_model[f][0] for f in foods_list_normalized]
foods_list_preprocessed = list(set(foods_list_preprocessed))

Load the word embedding for each food in the list of sample foods, and save to a dictionary.

In [ ]:
foods_vecs = dict()

word_vectors = wine_word2vec_model.wv
for f in foods_list_preprocessed:
    try:
        food_vec = word_vectors[f]
        foods_vecs[f] = food_vec
    except:
        continue

Now, we can define the nonaroma embeddings + the weight embedding as the average of foods that represent each nonaroma characteristic.

In [ ]:
from scipy import spatial

core_tastes_revised = {'weight': ['heavy', 'cassoulet', 'cassoulet', 'full_bodied', 'thick', 'milk', 'fat', 'mincemeat', 'steak', 'bold', 'pizza', 'pasta', 'creamy', 'bread'],
                       'sweet': ['sweet', 'sugar', 'cake', 'mango', 'stevia'],
                       'acid': ['acid', 'sour', 'vinegar', 'yoghurt', 'cevich', 'cevich'],
                       'salt': ['salty', 'salty', 'parmesan', 'oyster', 'pizza', 'bacon', 'cured_meat', 'sausage', 'potato_chip'],
                       'piquant': ['spicy'],
                       'fat': ['fat', 'fried', 'creamy', 'cassoulet', 'foie_gras', 'buttery', 'cake', 'foie_gras', 'sausage', 'brie', 'carbonara'],
                       'bitter': ['bitter', 'kale']
                      }

average_taste_vecs = dict()
core_tastes_distances = dict()
for taste, keywords in core_tastes_revised.items():

    all_keyword_vecs = []
    for keyword in keywords:
        c_vec = word_vectors[keyword]
        all_keyword_vecs.append(c_vec)

    avg_taste_vec = np.average(all_keyword_vecs, axis=0)
    average_taste_vecs[taste] = avg_taste_vec

    taste_distances = dict()
    for k, v in foods_vecs.items():
        similarity = 1- spatial.distance.cosine(avg_taste_vec, v)
        taste_distances[k] = similarity

    core_tastes_distances[taste] = taste_distances

We can now find out which foods most and least resemble each nonaroma.

In [ ]:
food_nonaroma_infos = dict()
# for each core taste, identify the food item that is farthest and closest. We will need this to create a normalized scale between 0 and 1
for key, value in core_tastes_revised.items():
    dict_taste = dict()
    farthest = min(core_tastes_distances[key], key=core_tastes_distances[key].get)
    farthest_distance = core_tastes_distances[key][farthest]
    closest = max(core_tastes_distances[key], key=core_tastes_distances[key].get)
    closest_distance = core_tastes_distances[key][closest]
    print(key, farthest, closest)
    dict_taste['farthest'] = farthest_distance
    dict_taste['closest'] = closest_distance
    dict_taste['average_vec'] = average_taste_vecs[key]
    food_nonaroma_infos[key] = dict_taste

Now, let's save the average embedding for each nonaroma, as well as the minimum and maximum distance to each nonaroma embedding - we will use these to scale the nonaroma scalars that we obtain for any foods we try to pair wine with.

In [ ]:
food_nonaroma_infos_df = pd.DataFrame(food_nonaroma_infos).T
food_nonaroma_infos_df.to_csv('average_nonaroma_vectors.csv')

We have all the pieces we need to build our wine recommendations. We will continue with this in a separate notebook.